In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np

In [18]:
dfElche = pd.read_csv('../Procesamiento/Elche-limpio.csv', sep=';')
dfOrihuela = pd.read_csv('../Procesamiento/Orihuela-limpio.csv', sep=';')
dfTorrevieja = pd.read_csv('../Procesamiento/Torrevieja-limpio.csv', sep=';')

df_combined = pd.concat([dfElche, dfOrihuela, dfTorrevieja], ignore_index=True)

df_combined.tail(10)

,Unnamed: 0,COD_ESTACION,NOM_ESTACION,FECHA,HORA,SO2,CO,NO,NO2,NOx,...,BkFA,Cd,DahA,FA,HMN,H2S,IcdP,Ni,Pb,PST
48230,48230,3133002,TORREVIEJA,2024-11-30,14,NaN,NaN,4.0,9.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48231,48231,3133002,TORREVIEJA,2024-11-30,15,NaN,NaN,3.0,9.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48232,48232,3133002,TORREVIEJA,2024-11-30,16,NaN,NaN,3.0,13.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48233,48233,3133002,TORREVIEJA,2024-11-30,17,NaN,NaN,3.0,16.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48234,48234,3133002,TORREVIEJA,2024-11-30,18,NaN,NaN,6.0,40.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48235,48235,3133002,TORREVIEJA,2024-11-30,19,NaN,NaN,16.0,59.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48236,48236,3133002,TORREVIEJA,2024-11-30,20,NaN,NaN,35.0,57.0,111.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48237,48237,3133002,TORREVIEJA,2024-11-30,21,NaN,NaN,21.0,47.0,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48238,48238,3133002,TORREVIEJA,2024-11-30,22,NaN,NaN,13.0,30.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48239,48239,3133002,TORREVIEJA,2024-11-30,23,NaN,NaN,9.0,21.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Veo la cantidad de nulos

In [19]:
df_combined.isnull().sum()

Unnamed: 0          0
COD_ESTACION        0
NOM_ESTACION        0
FECHA               0
HORA                0
SO2             31081
CO              31130
NO              16059
NO2             16059
NOx             16059
O3               1370
PM10            32392
PM2.5           32392
PM1             48240
NH3             48240
C6H6            48240
C7H8            48240
C8H10           48240
Direc.          31078
H.Rel.           2180
Precip.         31047
Pres.           31120
R.Sol.          31103
Ruido           48240
Temp.            2236
UV-B            48240
Veloc.          31397
Veloc.max.      48240
As              48240
BaA             48240
BaP             48240
BbFA            48240
BjFA            48240
BkFA            48240
Cd              48240
DahA            48240
FA              48240
HMN             48240
H2S             48240
IcdP            48240
Ni              48240
Pb              48240
PST             48240
dtype: int64

Relleno las columnas con menos de mil nulos

In [20]:
df_combined = df_combined.apply(lambda col: col.astype('category').cat.codes if col.dtypes == 'object' or col.dtypes.name == 'category' else col)
df_combined

,Unnamed: 0,COD_ESTACION,NOM_ESTACION,FECHA,HORA,SO2,CO,NO,NO2,NOx,...,BkFA,Cd,DahA,FA,HMN,H2S,IcdP,Ni,Pb,PST
0,0,3065006,0,0,0,4.0,0,1.0,14.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3065006,0,0,1,4.0,0,1.0,18.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3065006,0,0,2,4.0,0,1.0,11.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3065006,0,0,3,4.0,0,1.0,10.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,3065006,0,0,4,4.0,0,1.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48235,48235,3133002,2,669,19,NaN,-1,16.0,59.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48236,48236,3133002,2,669,20,NaN,-1,35.0,57.0,111.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48237,48237,3133002,2,669,21,NaN,-1,21.0,47.0,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48238,48238,3133002,2,669,22,NaN,-1,13.0,30.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
columnas_menos_nulos = df_combined.columns[df_combined.isnull().sum() < 400000]

for col in columnas_menos_nulos:
    df_combined[col].fillna(df_combined[col].median(), inplace=True)

C:\Users\javie\AppData\Local\Temp\ipykernel_15972\3271397801.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_combined[col].fillna(df_combined[col].median(), inplace=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_15972\3271397801.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [22]:
df_combined.isnull().sum()

Unnamed: 0          0
COD_ESTACION        0
NOM_ESTACION        0
FECHA               0
HORA                0
SO2                 0
CO                  0
NO                  0
NO2                 0
NOx                 0
O3                  0
PM10                0
PM2.5               0
PM1             48240
NH3             48240
C6H6            48240
C7H8            48240
C8H10           48240
Direc.              0
H.Rel.              0
Precip.             0
Pres.               0
R.Sol.              0
Ruido           48240
Temp.               0
UV-B            48240
Veloc.              0
Veloc.max.      48240
As              48240
BaA             48240
BaP             48240
BbFA            48240
BjFA            48240
BkFA            48240
Cd              48240
DahA            48240
FA              48240
HMN             48240
H2S             48240
IcdP            48240
Ni              48240
Pb              48240
PST             48240
dtype: int64

Elimino las que tengan mas nulos 

In [23]:
nulos_por_columna = df_combined.isnull().sum()
columnas_mas_1000_nulos = nulos_por_columna[nulos_por_columna >= 400000].index

df_combined.drop(columns=columnas_mas_1000_nulos, inplace=True)

In [24]:
df_combined.isnull().sum()

Unnamed: 0          0
COD_ESTACION        0
NOM_ESTACION        0
FECHA               0
HORA                0
SO2                 0
CO                  0
NO                  0
NO2                 0
NOx                 0
O3                  0
PM10                0
PM2.5               0
PM1             48240
NH3             48240
C6H6            48240
C7H8            48240
C8H10           48240
Direc.              0
H.Rel.              0
Precip.             0
Pres.               0
R.Sol.              0
Ruido           48240
Temp.               0
UV-B            48240
Veloc.              0
Veloc.max.      48240
As              48240
BaA             48240
BaP             48240
BbFA            48240
BjFA            48240
BkFA            48240
Cd              48240
DahA            48240
FA              48240
HMN             48240
H2S             48240
IcdP            48240
Ni              48240
Pb              48240
PST             48240
dtype: int64

In [25]:
y = df_combined['SO2']
X = df_combined.drop(['SO2'],axis=1)

In [26]:
y

0        4.0
1        4.0
2        4.0
3        4.0
4        4.0
        ... 
48235    4.0
48236    4.0
48237    4.0
48238    4.0
48239    4.0
Name: SO2, Length: 48240, dtype: float64

In [27]:
X

,Unnamed: 0,COD_ESTACION,NOM_ESTACION,FECHA,HORA,CO,NO,NO2,NOx,O3,...,BkFA,Cd,DahA,FA,HMN,H2S,IcdP,Ni,Pb,PST
0,0,3065006,0,0,0,0,1.0,14.0,14.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3065006,0,0,1,0,1.0,18.0,18.0,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3065006,0,0,2,0,1.0,11.0,11.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3065006,0,0,3,0,1.0,10.0,10.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,3065006,0,0,4,0,1.0,7.0,7.0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48235,48235,3133002,2,669,19,-1,16.0,59.0,83.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48236,48236,3133002,2,669,20,-1,35.0,57.0,111.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48237,48237,3133002,2,669,21,-1,21.0,47.0,79.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48238,48238,3133002,2,669,22,-1,13.0,30.0,49.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
from sklearn.model_selection import train_test_split 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [29]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [30]:
model.score(X_test,y_test)

0.9385364842454395

Usando Dos algoritmos y viendo con graficas cada uno Automatizado

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Leer los archivos CSV
dfElche = pd.read_csv('../Procesamiento/Elche-limpio.csv', sep=';')
dfOrihuela = pd.read_csv('../Procesamiento/Orihuela-limpio.csv', sep=';')
dfTorrevieja = pd.read_csv('../Procesamiento/Torrevieja-limpio.csv', sep=';')

# Concatenar los DataFrames
df_combined = pd.concat([dfElche, dfOrihuela, dfTorrevieja], ignore_index=True)

# Mostrar las últimas 10 filas
print(df_combined.tail(10))

# Comprobar valores nulos
print(df_combined.isnull().sum())

# Convertir columnas de tipo 'object' o 'category' a códigos numéricos
df_combined = df_combined.apply(lambda col: col.astype('category').cat.codes if col.dtypes == 'object' or col.dtypes.name == 'category' else col)

# Imprimir DataFrame
print(df_combined)

# Seleccionar columnas con menos de 400,000 valores nulos
columnas_menos_nulos = df_combined.columns[df_combined.isnull().sum() < 400000]

# Rellenar valores nulos con la mediana
for col in columnas_menos_nulos:
    df_combined[col].fillna(df_combined[col].median(), inplace=True)

# Comprobar de nuevo los valores nulos
print(df_combined.isnull().sum())

# Eliminar columnas con más de 400,000 valores nulos
nulos_por_columna = df_combined.isnull().sum()
columnas_mas_1000_nulos = nulos_por_columna[nulos_por_columna >= 400000].index
df_combined.drop(columns=columnas_mas_1000_nulos, inplace=True)

# Comprobar si hay más valores nulos
print(df_combined.isnull().sum())

# Separar la variable dependiente (SO2) de las independientes
y = df_combined['SO2']
X = df_combined.drop(['SO2'], axis=1)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Inicializar los modelos
rf_model = RandomForestClassifier()
lr_model = LogisticRegression(max_iter=1000)


# Entrenar los modelos
rf_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)

# Hacer predicciones
rf_predictions = rf_model.predict(X_test)
lr_predictions = lr_model.predict(X_test)


# Evaluar precisión de ambos modelos
rf_accuracy = accuracy_score(y_test, rf_predictions)
lr_accuracy = accuracy_score(y_test, lr_predictions)


# Mostrar precisión de ambos modelos
print(f"Precisión de RandomForest: {rf_accuracy * 100:.2f}%")
print(f"Precisión de LogisticRegression: {lr_accuracy * 100:.2f}%")

# Graficar resultados
models = ['RandomForest', 'Logistic Regression']
accuracies = [rf_accuracy, lr_accuracy]

# Configuración de las gráficas
plt.figure(figsize=(8, 5))
sns.barplot(x=models, y=accuracies, palette='viridis')
plt.title('Comparación de Precisión de Modelos')
plt.xlabel('Modelo')
plt.ylabel('Precisión')
plt.ylim(0, 1)
plt.show()


       Unnamed: 0  COD_ESTACION NOM_ESTACION       FECHA  HORA  SO2   CO  \
48230       48230       3133002   TORREVIEJA  2024-11-30    14  NaN  NaN   
48231       48231       3133002   TORREVIEJA  2024-11-30    15  NaN  NaN   
48232       48232       3133002   TORREVIEJA  2024-11-30    16  NaN  NaN   
48233       48233       3133002   TORREVIEJA  2024-11-30    17  NaN  NaN   
48234       48234       3133002   TORREVIEJA  2024-11-30    18  NaN  NaN   
48235       48235       3133002   TORREVIEJA  2024-11-30    19  NaN  NaN   
48236       48236       3133002   TORREVIEJA  2024-11-30    20  NaN  NaN   
48237       48237       3133002   TORREVIEJA  2024-11-30    21  NaN  NaN   
48238       48238       3133002   TORREVIEJA  2024-11-30    22  NaN  NaN   
48239       48239       3133002   TORREVIEJA  2024-11-30    23  NaN  NaN   

         NO   NO2    NOx  ...  BkFA  Cd  DahA  FA  HMN  H2S  IcdP  Ni  Pb  PST  
48230   4.0   9.0   15.0  ...   NaN NaN   NaN NaN  NaN  NaN   NaN NaN NaN  NaN  
4

C:\Users\javie\AppData\Local\Temp\ipykernel_15972\1181484005.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_combined[col].fillna(df_combined[col].median(), inplace=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_15972\1181484005.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

Sacando el ml a un archivo pickle

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# Leer los archivos CSV
dfElche = pd.read_csv('../Procesamiento/Elche-limpio.csv', sep=';')
dfOrihuela = pd.read_csv('../Procesamiento/Orihuela-limpio.csv', sep=';')
dfTorrevieja = pd.read_csv('../Procesamiento/Torrevieja-limpio.csv', sep=';')

# Concatenar los DataFrames
df_combined = pd.concat([dfElche, dfOrihuela, dfTorrevieja], ignore_index=True)

# Mostrar las últimas 10 filas
print(df_combined.tail(10))

# Comprobar valores nulos
print(df_combined.isnull().sum())

# Convertir columnas de tipo 'object' o 'category' a códigos numéricos
df_combined = df_combined.apply(lambda col: col.astype('category').cat.codes if col.dtypes == 'object' or col.dtypes.name == 'category' else col)

# Imprimir DataFrame
print(df_combined)

# Seleccionar columnas con menos de 400,000 valores nulos
columnas_menos_nulos = df_combined.columns[df_combined.isnull().sum() < 400000]

# Rellenar valores nulos con la mediana
for col in columnas_menos_nulos:
    df_combined[col].fillna(df_combined[col].median(), inplace=True)

# Comprobar de nuevo los valores nulos
print(df_combined.isnull().sum())

# Eliminar columnas con más de 400,000 valores nulos
nulos_por_columna = df_combined.isnull().sum()
columnas_mas_1000_nulos = nulos_por_columna[nulos_por_columna >= 400000].index
df_combined.drop(columns=columnas_mas_1000_nulos, inplace=True)

# Comprobar si hay más valores nulos
print(df_combined.isnull().sum())

# Separar la variable dependiente (SO2) de las independientes
y = df_combined['SO2']
X = df_combined.drop(['SO2'], axis=1)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Inicializar los modelos
rf_model = RandomForestClassifier()
lr_model = LogisticRegression(max_iter=1000)

# Entrenar los modelos
rf_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)

# Hacer predicciones
rf_predictions = rf_model.predict(X_test)
lr_predictions = lr_model.predict(X_test)

# Evaluar precisión de ambos modelos
rf_accuracy = accuracy_score(y_test, rf_predictions)
lr_accuracy = accuracy_score(y_test, lr_predictions)

# Mostrar precisión de ambos modelos
print(f"Precisión de RandomForest: {rf_accuracy * 100:.2f}%")
print(f"Precisión de LogisticRegression: {lr_accuracy * 100:.2f}%")

# Guardar el modelo con mejor puntaje en un archivo pickle
best_model = rf_model if rf_accuracy > lr_accuracy else lr_model
target_name = 'SO2'
pickle_filename = f"{target_name}_model.pkl"

with open(pickle_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Modelo con mejor puntaje ({'RandomForest' if best_model == rf_model else 'Logistic Regression'}) guardado como '{pickle_filename}'.")


       Unnamed: 0  COD_ESTACION NOM_ESTACION       FECHA  HORA  SO2   CO  \
48230       48230       3133002   TORREVIEJA  2024-11-30    14  NaN  NaN   
48231       48231       3133002   TORREVIEJA  2024-11-30    15  NaN  NaN   
48232       48232       3133002   TORREVIEJA  2024-11-30    16  NaN  NaN   
48233       48233       3133002   TORREVIEJA  2024-11-30    17  NaN  NaN   
48234       48234       3133002   TORREVIEJA  2024-11-30    18  NaN  NaN   
48235       48235       3133002   TORREVIEJA  2024-11-30    19  NaN  NaN   
48236       48236       3133002   TORREVIEJA  2024-11-30    20  NaN  NaN   
48237       48237       3133002   TORREVIEJA  2024-11-30    21  NaN  NaN   
48238       48238       3133002   TORREVIEJA  2024-11-30    22  NaN  NaN   
48239       48239       3133002   TORREVIEJA  2024-11-30    23  NaN  NaN   

         NO   NO2    NOx  ...  BkFA  Cd  DahA  FA  HMN  H2S  IcdP  Ni  Pb  PST  
48230   4.0   9.0   15.0  ...   NaN NaN   NaN NaN  NaN  NaN   NaN NaN NaN  NaN  
4

C:\Users\javie\AppData\Local\Temp\ipykernel_15972\3603665226.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_combined[col].fillna(df_combined[col].median(), inplace=True)
C:\Users\javie\AppData\Local\Temp\ipykernel_15972\3603665226.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values